In [1]:
import pandas as pd
import math
from typing import List
from datetime import datetime
from sqlalchemy import MetaData, Table, create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import Integer

In [2]:
import sys
import os

In [3]:
engine = create_engine('sqlite:///pare.db')

In [4]:
sys.path.append(os.path.join(os.path.dirname('__file__'), 'models'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'scripts'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'config'))

In [5]:
from config.params import DEBUG

In [6]:
from scripts.utils import random_item, random_date, random_date_between, random_int, years_between

In [7]:
from models.activity import Activity, TypeOfActivity

In [8]:
from scripts.constants import (JOINED_START_DATE, 
                                JOINED_END_DATE,)

start_date = JOINED_START_DATE
end_date = JOINED_END_DATE

In [9]:
partner_df = pd.read_sql('partners', con=engine)
partner_list = partner_df.to_dict(orient='records')

In [10]:
activity_list = [
    {"name": "Training sesssion", "activity_type": TypeOfActivity.capacity_building.value},
    {"name": "Input distribution", "activity_type": TypeOfActivity.distrib_intrant.value},
    {"name": "Reunion de plaidoyer", "activity_type": TypeOfActivity.reunion_de_plaidoyer.value},
    {"name": "Liaison avec le marché", "activity_type": TypeOfActivity.liaison_avec_le_marche.value},
    {"name": "Autre", "activity_type": TypeOfActivity.liaison_avec_le_marche.autre},
]

In [11]:
TypeOfActivity.capacity_building.value

'Formation/renforcement des capacités'

In [12]:
#partner_df.head()

In [13]:
desc_text = [
    "Video provides a powerful way to help you prove your point. When you click Online Video, you can paste in the embed code for the video you want to add. You can also type a keyword to search online for the video that best fits your document. To make your document look professionally produced, Word provides header, footer, cover page, and text box designs that complement each other.",
    "For example, you can add a matching cover page, header, and sidebar. Click Insert and then choose the elements you want from the different galleries. Themes and styles also help keep your document coordinated. When you click Design and choose a new Theme, the pictures, charts, and SmartArt graphics change to match your new theme.",
    "When you apply styles, your headings change to match the new theme. Save time in Word with new buttons that show up where you need them. To change the way a picture fits in your document, click it and a button for layout options appears next to it. When you work on a table, click where you want to add a row or a column, and then click the plus sign.",
    "Reading is easier, too, in the new Reading view. You can collapse parts of the document and focus on the text you want. If you need to stop reading before you reach the end, Word remembers where you left off - even on another device. Video provides a powerful way to help you prove your point.",
    "When you click Online Video, you can paste in the embed code for the video you want to add. You can also type a keyword to search online for the video that best fits your document. To make your document look professionally produced, Word provides header, footer, cover page, and text box designs that complement each other. For example, you can add a matching cover page, header, and sidebar.",
    "Click Insert and then choose the elements you want from the different galleries. Themes and styles also help keep your document coordinated. When you click Design and choose a new Theme, the pictures, charts, and SmartArt graphics change to match your new theme. When you apply styles, your headings change to match the new theme.",
    "Save time in Word with new buttons that show up where you need them. To change the way a picture fits in your document, click it and a button for layout options appears next to it. When you work on a table, click where you want to add a row or a column, and then click the plus sign. Reading is easier, too, in the new Reading view.",
    "You can collapse parts of the document and focus on the text you want. If you need to stop reading before you reach the end, Word remembers where you left off - even on another device. Video provides a powerful way to help you prove your point. When you click Online Video, you can paste in the embed code for the video you want to add.",
    "You can also type a keyword to search online for the video that best fits your document. To make your document look professionally produced, Word provides header, footer, cover page, and text box designs that complement each other. For example, you can add a matching cover page, header, and sidebar. Click Insert and then choose the elements you want from the different galleries.",
    "Themes and styles also help keep your document coordinated. When you click Design and choose a new Theme, the pictures, charts, and SmartArt graphics change to match your new theme. When you apply styles, your headings change to match the new theme. Save time in Word with new buttons that show up where you need them.",
]

In [14]:
# generate activity



def generate_activity(activity_list, partner_list, desc_text, pk) -> Activity:
    rand_activity = random_item(activity_list)
    desc = random_item(desc_text)
    partner = random_item(partner_list)
    max_participant = 1000
    min_participant = 10
    total_male_participant = random_int(min_participant, max_participant)
    total_female_participant = random_int(min_participant, max_participant)
    total_participant = total_male_participant + total_female_participant
    
    total_male_youth_participant =  random_int(0, total_male_participant)
    total_female_youth_participant = random_int(0, total_female_participant)
    
    total_youth = total_male_youth_participant +  total_female_youth_participant 

    activity = Activity(
        id=pk,
        name =  rand_activity.get("name"),
        community_id = partner["community_id"],
        partner_id = partner["id"],
        activity_type = rand_activity.get("activity_type"),
        total_participant = total_participant,
        total_male_participant = total_male_participant,
        total_female_participant = total_female_participant,
        total_male_youth_participant = total_male_youth_participant,
        total_female_youth_participant = total_female_youth_participant,
        total_youth_participant =  total_youth,
        description = desc,
        date = random_date_between(start_date, end_date)
    )
    #create the activity with a random name and a type

    #get a random start and end date

    return activity  
    

In [15]:
#partner = random_item(partner_list)

In [16]:
activities = []

for i in range(500):
    activities.append(generate_activity(activity_list, partner_list, desc_text, i).dict())

In [17]:
activity_df = pd.DataFrame(activities)

In [18]:
activity_df.head()

,id,name,community_id,partner_id,activity_type,total_participant,total_male_participant,total_female_participant,total_male_youth_participant,total_female_youth_participant,total_youth_participant,description,date
0,0,Autre,196,97,Autre,1700,882,818,642,179,821,You can also type a keyword to search online f...,2024-03-12
1,1,Reunion de plaidoyer,179,92,Réunion de plaidoyer,228,209,19,78,8,86,"Reading is easier, too, in the new Reading vie...",2027-07-12
2,2,Reunion de plaidoyer,158,22,Réunion de plaidoyer,951,662,289,285,82,367,Themes and styles also help keep your document...,2024-05-29
3,3,Liaison avec le marché,157,211,Liaison avec le marché,1003,41,962,22,210,232,Save time in Word with new buttons that show u...,2027-08-09
4,4,Input distribution,16,107,Distribution d'intrants,1663,982,681,461,192,653,"For example, you can add a matching cover page...",2025-05-30


In [19]:
#activity_df.to_csv("./outputs/activities.csv")

In [20]:
activity_df.to_sql('activities', con=engine, if_exists='replace', index=False, dtype={'id': Integer, 'partner_id': Integer, 'community_id': Integer})

500